In [ ]:
import json
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np
import keras.backend as K
import cv2
import lime 
from skimage.segmentation import quickshift,mark_boundaries
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import skimage.io

In [ ]:
import tensorflow as tf
import time

In [ ]:


import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)


In [ ]:
# Charger le modèle
model = load_model("/kaggle/input/bootstrap-data/ModelSjogrenn(2).h5")



In [ ]:
model.summary()

In [ ]:
model

In [ ]:


x=np.load("/kaggle/input/bootstrap-data/x_test(2).npy")
y=np.load("/kaggle/input/bootstrap-data/y_test(2).npy")



In [ ]:
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

y_pred2 = model.predict(x)
y_pred_classes2 = np.where(y_pred2>0.5,1.,0)

# Calculer la matrice de confusion
cm = confusion_matrix(y_true=y, y_pred=y_pred_classes2)
f,ax = plt.subplots(figsize=(5,5))
sns.heatmap(cm, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
print(cm)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred_classes2))

In [ ]:

# Calculez la sensibilité, la spécificité, le FPR et le FNR à partir de la matrice de confusion
TN, FP, FN, TP = cm.ravel()
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# Calculez le LR+ et le LR- à partir de la sensibilité et de la spécificité
lr_plus = sensitivity / (1 - specificity)
lr_minus = (1 - sensitivity) / specificity

# Afficher les résultats
print("Matrice de confusion : ")
print(cm)
print("Sensibilité (TPR) : ", sensitivity)
print("Spécificité (TNR) : ", specificity)
print("Faux positifs (FPR) : ", FPR)
print("Faux négatifs (FNR) : ", FNR)
print("Rapport de vraisemblance positive (LR+) : ", lr_plus)
print("Rapport de vraisemblance négative (LR-) : ", lr_minus)



In [ ]:
y_true = y  # Vraies étiquettes (0 pour négatif, 1 pour positif)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

# Calcul de la courbe ROC
fpr, tpr, thresholds = roc_curve(y, y_pred_classes2)

# Création du graphique ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()



In [ ]:
from sklearn.metrics import roc_auc_score


# Calcul de l'aire sous la courbe ROC (ROC-AUC)
roc_auc = roc_auc_score(y, y_pred_classes2)

print("ROC-AUC : {:.2f}".format(roc_auc))

In [ ]:
from PIL import Image
chemin_image = "/kaggle/input/sjdata/DATA_(512,512)/sj_data/019.jpg"
img=Image.open(chemin_image)

In [ ]:
# Convert to RGB mode
img_resized_rgb = img.convert("RGB")


In [ ]:

img2 = np.array(img_resized_rgb)


In [ ]:
img2.shape

In [ ]:
im = np.array(img)

In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [ ]:
plt.imshow(im / 2 + 0.5,cmap="gray")

In [ ]:
from lime.wrappers.scikit_image import SegmentationAlgorithm
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('quickshift', kernel_size=1, max_dist=200, ratio=0.2)

In [ ]:
image = np.expand_dims(im, axis=0)

In [ ]:
image.shape

In [ ]:
preds =model.predict(x[0:10])
pred_classes= np.where(preds>0.5,1.,0)

In [ ]:
preds =model.predict(image)
pred_classes= np.where(preds>0.5,1.,0)

In [ ]:
image.shape

In [ ]:
mage=rgb2gray(img2)
mage =np.expand_dims(mage, axis=0)

In [ ]:
mage.shape

In [ ]:
def predict_fn(images):
    s= np.array(img)
    s= np.expand_dims(s, axis=0)
    preds = model.predict(s)
    pred_class = np.where(preds>0.5,1.,0)
    return pred_class

def predict_fn(x):
    gray_image = rgb2gray(x)
    gray_image = gray_image.astype(np.float32)
    gray_image = np.expand_dims(im, axis=0)
    print(gray_image.shape)
    preds = model.predict(gray_image)
   
    return preds

In [ ]:
predict_fn(gray2rgb(img))

In [ ]:
# Create the Lime explanation for the image
explanation = explainer.explain_instance(gray2rgb(img),predict_fn,labels=(1,),num_features=50, top_labels=10, hide_color=0, num_samples=100, segmentation_fn=segmenter)

In [ ]:
explanation

In [ ]:

# Get the top explanations
top_explanations = explanation.top_labels[0]

In [ ]:
available_labels = explanation.local_exp.keys()
print(available_labels)

In [ ]:
temp, mask = explanation.get_image_and_mask(0, positive_only=True, num_features=1, hide_rest=True)

In [ ]:
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

## part2 with linear_regressor and perturbations

In [ ]:
Xi = skimage.transform.resize(img2, (512,512)) # resizing image
skimage.io.imshow(Xi)# Show image 
Xi = (Xi - 0.5)*2

In [ ]:
superpixels = quickshift(Xi, kernel_size=4,max_dist=200, ratio=0.2) #the higher the kernel size,max_dist, the fewer are the clusters

In [ ]:
#Segments image using quickshift clustering
num_superpixels = np.unique(superpixels).shape[0]
print("The number of super pixels generated")
num_superpixels

In [ ]:
skimage.io.imshow(mark_boundaries(Xi/2+0.5, superpixels))

In [ ]:
np.random.seed(120)
num_perturb = 100
#150 perturbations
perturbations = np.random.binomial(1, 0.5, size=(num_perturb, num_superpixels))# The size corresponds to the (no. of perturbations x no. of superpixels) in the image.
print(" Here, '1' represent ON(active) superpixel and '0' represents OFF.")
perturbations[0]

In [ ]:


import copy



In [ ]:
def perturb_image(img,perturbation,segments): #takes in the parameters : raw image, perturbation vector and superpixels generated
  active_pixels = np.where(perturbation == 1)[0]
  mask = np.zeros(segments.shape)
  for active in active_pixels:
      mask[segments == active] = 1 
  perturbed_image = copy.deepcopy(img)
  perturbed_image = perturbed_image*mask[:,:,np.newaxis]
  return perturbed_image #returns the perturbed image



In [ ]:
print("The perturbed image")
skimage.io.imshow(perturb_image(Xi/2+0.5,perturbations[0],superpixels)) 

In [ ]:
pred = model.predict(image)

In [ ]:
predictions = []
for pert in perturbations:
  perturbed_img = perturb_image(Xi,pert,superpixels)
  # Assuming your image is stored in a variable called 'image'
  gray_image = rgb2gray(perturbed_img)
  gray_image = gray_image.astype(np.float32)
  gray_image = np.expand_dims(im, axis=0)
  print(gray_image.shape)
  pred = model.predict(gray_image)
  predictions.append(pred)


In [ ]:
import sklearn.metrics

In [ ]:
original_image = np.ones(num_superpixels)[np.newaxis,:] #Perturbation with all superpixels enabled 
#The distance between each randomly generated perturnation and the image being explained is computed using the cosine distance. 
distances = sklearn.metrics.pairwise_distances(perturbations,original_image, metric='cosine').ravel()#ravel() function is used to create a contiguous flattened array.
#distances

In [ ]:
kernel_width = 0.25
weights = np.sqrt(np.exp(-(distances**2)/kernel_width**2)) #Kernel function
weights.shape

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
predictions = np.array(predictions)

In [ ]:
top_pred_classes=(0,1)

In [ ]:
simpler_model = LinearRegression()
simpler_model.fit(X=perturbations, y=predictions[:,:,top_pred_classes[0]], sample_weight=weights) # the top predicted class is to be explained
coeff = simpler_model.coef_[0] 
coeff.shape # Each coefficient in the linear model corresponds to one superpixel in the segmented image.

In [ ]:
num_top_features = 100
top_features = np.argsort(coeff)[-num_top_features:] 
#Now we need to sort the coefficients to figure out which are the superpixels that have larger coefficients (magnitude) for the prediction of egyptian cat. 
top_features

In [ ]:
mask = np.zeros(num_superpixels) #The less relevant pixels are black and only the top superpixels are activated.
mask[top_features]= True 
skimage.io.imshow(perturb_image(Xi/2+0.5,mask,superpixels) )

In [ ]:
image.shape

In [ ]:

import matplotlib.pyplot as plt

# Taux de faux positifs (FPR) et taux de faux négatifs (FNR)
fpr = [0.005681818181818182]  # Exemple de FPR
fnr = [0.01086956521739130]  # Exemple de FNR

# Création du graphique ROC
roc_values = [1 - fnr[i] for i in range(len(fpr))]  # Calcul du taux de vrais positifs (1 - FNR)
plt.figure(figsize=(8, 6))
plt.plot(fpr, roc_values, marker='o', color='darkorange', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (1 - FNR)')
plt.title('Courbe ROC')
plt.show()
